In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime

In [ ]:
# load data
df = pd.read_csv('picasso_data.csv')
cols = ['auc_end_date', 'auctioneer', 'auctioneer_address', 'bid_history',
       'est_price', 'item_description', 'item_title', 'item_url',
       'sold_price']
df.columns = cols

In [ ]:
# nan
df = df.replace(to_replace='None', value=np.nan).dropna().reset_index(drop=True)

In [ ]:
# auc_end_weekday
df['auc_end_weekday'] = (df.auc_end_date.str.strip()
                          .str.split(' on ',expand=True)[1]
                          .str.strip().str.split(', ', 1, expand=True)[0]
                         )

In [ ]:
# auc_end_date
df['auc_end_date_lean'] = (df.auc_end_date.str.strip()
                            .str.split(' on ',expand=True)[1]
                            .str.strip().str.split(', ', 1, expand=True)[1]
                           )


In [ ]:
# price functions
def get_price_value(x):
    num_thousand = x[re.search('\d', x).start():]
    num_string = num_thousand.replace(',','')
    num_float = float(num_string.strip())
    num_int = int(num_float)
    return num_int

def get_price_currency(x):
    return (x[:re.search('\d', x).start()]
            .strip()
           )

In [ ]:
# get price
df['sold_price_value'] = (df.sold_price.str.strip()
                           .str.split(' ',expand=True)[2]
                           .apply(get_price_value)
                          )

In [ ]:
# get currency
df['sold_price_currency'] = (df.sold_price.str.strip()
                              .str.split(' ',expand=True)[2]
                              .apply(get_price_currency)
                             )

In [ ]:
# check currencies
df.sold_price_currency.value_counts()

In [ ]:
df['est_low_value'] = (df\.est_price.str.strip()
                        .str.split(' ', expand=True)[1]
                        .apply(get_price_value)
                       )

df['est_low_currency'] = (df.est_price
                           .str.strip().str.split(' ', expand=True)[1]
                           .apply(get_price_currency)
                          )

df['est_high_value'] = (df.est_price.str.strip()
                        .str.split(' ', expand=True)[3]
                        .apply(get_price_value)
                        )
df['est_high_currency'] = (df.est_price
                           .str.strip().str.split(' ', expand=True)[3]
                           .apply(get_price_currency)
                          )

df['bid_start_value'] = [get_price_value(x[-1]) for x in df.bid_history]

df['bid_start_currency'] = [get_price_currency(x[-1]) for x in df.bid_history]

In [ ]:
# map the dollor signs used to the standard currency code
cur_ref = [
    ['$', 'USD'],
    ['€', 'EUR'],
    ['£', 'GBP'],
    ['CHF', 'CHF'],
    ['CA$', 'CAD'],
    ['A$', 'AUD'],
    ['PLN', 'PLN'],
    ['NZ$', 'NZD'],
    ['NOK', 'NOK'],
    ['NT$', 'TWD'],
    ['MX$', 'MXN'],
    ['HK$', 'HKD']
]

In [ ]:
# replace the currency signs to standard code in the df
for i in range(len(cur_ref)):
    df['sold_price_currency'] = df['sold_price_currency'].replace(cur_ref[i][0], cur_ref[i][1])
    df['est_low_currency'] = df['est_low_currency'].replace(cur_ref[i][0], cur_ref[i][1])
    df['est_high_currency'] = df['est_high_currency'].replace(cur_ref[i][0], cur_ref[i][1])
    df['bid_start_currency'] = df['bid_start_currency'].replace(cur_ref[i][0], cur_ref[i][1])

In [ ]:
# convert source string to datetime datatype for future timedelta use
df['auc_end_date_lean'] = df['auc_end_date_lean'].apply(lambda x: datetime.strptime(x.strip(), '%b %d, %Y'))

In [ ]:
# make a string of date for future api use
df['auc_end_data_str'] = df['auc_end_date_lean'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [ ]:
# check the currency consistency for each item
for i in range(len(df)):
    if df['sold_price_currency'][i] == df['est_low_currency'][i] == df['est_high_currency'][i] == df['bid_start_currency'][i] is False:
        print(i)

In [ ]:
import json
import requests

In [ ]:
# use df['sold_price_currency'] for exchange rate reference
def get_rate(cur, date):
    apiurl = 'http://apilayer.net/api/historical'
    key = 'a0e1e75cba889e9457d43ebaa58e39fe'
    url = apiurl + '?access_key=' + key + '&date=' + date + '&currencies=' + cur
    if cur == 'USD':
        return 1.0
    else:
        response = requests.get(url)
        json_data = json.loads(response.text)
        return json_data['quotes']['USD' + cur]

In [ ]:
# find currency denominators
df['currency_denominator'] = [get_rate(df['sold_price_currency'][i], df['auc_end_data_str'][i]) for i in range(len(df))]

In [ ]:
# sold price in usd
df['sold_price_value_usd'] = [int(df['sold_price_value'][i] / df['currency_denominator'][i]) for i in range(len(df))]

In [ ]:
# estimate low in usd
df2['est_low_value_usd'] = [int(df['est_low_value'][i] / df['currency_denominator'][i]) for i in range(len(df))]

In [ ]:
# estimate high in usd
df['est_high_value_usd'] = [int(df['est_high_value'][i] / df['currency_denominator'][i]) for i in range(len(df))]

In [ ]:
# bid start in usd
df['bid_start_value_usd'] = [int(df['bid_start_value'][i] / df['currency_denominator'][i]) for i in range(len(df))]

In [ ]:
# calculate the days from auction to now
df['auc_to_now_days'] = df['auc_end_date_lean'].apply(lambda x: (datetime.now() - x).days)

In [ ]:
# get the auction country/region from auctioneer_address
df['auc_region'] = [x[-1].strip() for x in df['auctioneer_address']]

In [ ]:
# word counts of item title
df['item_title_word_count'] = [len(x.split()) for x in df['item_title']]

In [ ]:
# convert item_description from list to string
df['item_description'] = df['item_description'].apply(lambda x: ' '.join([str(i) for i in x]))

In [ ]:
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

In [ ]:
# remove the html tags
df['item_description'] = df['item_description'].apply(striphtml)

In [ ]:
# word counts of item description
df['item_description_word_count'] = [len(x.split()) for x in df['item_description']]

In [ ]:
# drop auctioneer_address
df.drop('auctioneer_address', axis=1, inplace=True)

# drop auctioneer
df.drop('auctioneer', axis=1, inplace=True)

In [ ]:
df.to_csv('picasso_data_before_text.csv', mode='a', index=False, header=False)

In [ ]:
# header = ['item_description', 'item_title', 'auc_end_weekday',
#        'sold_price_value_usd', 'est_low_value_usd', 'est_high_value_usd',
#        'bid_start_value_usd', 'auc_to_now_days', 'auc_region',
#        'item_title_word_count', 'item_description_word_count']